# Vector Fitting

Se p1 e p2 são um par complexo conjugado, então c1 e c2 também formar um par complexo conjugado, assim como d1 e d2. De forma com que $\theta$ passa a ser:
$$\theta = \begin{bmatrix} c_0 & \Re(c_1) & \Im(c_1) & \Re(d_1) & \Im(d_1) \end{bmatrix} $$
Quando isso ocorre, fazer uma substituição na matrix M de:
$$\frac{1}{S-p_1}$$
e
$$\frac{1}{S-p_2}$$
por:
$$\frac{1}{S-p_1}+\frac{1}{S-p_1^*}$$
e
$$\frac{j}{S-p_1}-\frac{j}{S-p_1^*}$$

# M
$$M =
\begin{bmatrix}
1 & \frac{1}{S_1-p_1}+\frac{1}{S_1-p_1^*} & \frac{j}{S_1-p_1}-\frac{j}{S_1-p_1^*} & -G(S_1)(\frac{1}{S_1-p_1}+\frac{1}{S_1-p_1^*}) & -G(S_1)(\frac{j}{S_1-p_1}-\frac{j}{S_1-p_1^*}) \\
1 & \frac{1}{S_2-p_1}+\frac{1}{S_2-p_1^*} & \frac{j}{S_2-p_1}-\frac{j}{S_2-p_1^*} & -G(S_2)(\frac{1}{S_2-p_1}+\frac{1}{S_2-p_1^*}) & -G(S_2)(\frac{j}{S_2-p_1}-\frac{j}{S_2-p_1^*}) \\
1 & \frac{1}{S_3-p_1}+\frac{1}{S_3-p_1^*} & \frac{j}{S_3-p_1}-\frac{j}{S_3-p_1^*} & -G(S_3)(\frac{1}{S_3-p_1}+\frac{1}{S_3-p_1^*}) & -G(S_3)(\frac{j}{S_3-p_1}-\frac{j}{S_3-p_1^*}) \\
1 & \frac{1}{S_4-p_1}+\frac{1}{S_4-p_1^*} & \frac{j}{S_4-p_1}-\frac{j}{S_4-p_1^*} & -G(S_4)(\frac{1}{S_4-p_1}+\frac{1}{S_4-p_1^*}) & -G(S_4)(\frac{j}{S_4-p_1}-\frac{j}{S_4-p_1^*}) \\
1 & \frac{1}{S_5-p_1}+\frac{1}{S_5-p_1^*} & \frac{j}{S_5-p_1}-\frac{j}{S_5-p_1^*} & -G(S_5)(\frac{1}{S_5-p_1}+\frac{1}{S_5-p_1^*}) & -G(S_5)(\frac{j}{S_5-p_1}-\frac{j}{S_5-p_1^*}) \\
\end{bmatrix}
$$

Com os d's e c's, vejo se os p's são reais ou pares complexos conjugados encontrando os zeros da função:
$$polos = zeros( 1+\sum\limits_{i=1}^{n}\frac{d_i}{S-p_i} ) $$
Encontrando os novos polos p para a nova iteração.

In [2]:
import numpy as np

In [ ]:

def lsqrt_polinomial(n, u, y):
    amostras = len(u)
    u = u.flatten()
    y = y.flatten()
    M = np.ones((amostras,n))
    for i in range(n):
        M[:,i] = u**i
    y = np.array(y)
    theta = np.linalg.inv(M.T.dot(M)).dot(M.T).dot(y)
    y_hat = np.sum(M*theta.T,axis=1)
    return theta, y_hat


In [45]:
def gen_m(u,y,i,p,params):
    if not i:
        return 1

    c = i < params/2
    d = not c
    real = i%2
    imag = not real

    i -= 1
    p, pp = p[i], p[i-1]
    complex = p.real < 100*p.imag

    print(f'i: {i}, complex: {complex}, real: {real}, imag: {imag}, c: {c}, d: {d}')

    if c and not complex:
        return 1/(u-p)

    if c and complex and real:
        return 1/(u-p)+1/(u-p.conj())

    if c and complex and imag:
        return 1j/(u-pp)-1j/(u-pp.conj())

    if d and not complex:
        return -y*1/(u-p)

    if d and complex and real:
        return -y*(1/(u-p)+1/(u-p.conj()))

    if d and complex and imag:
        return -y*(1j/(u-pp)-1j/(u-pp.conj()))


In [ ]:
def lsqr_complex_polinomial_vf(u, y, n=2, p=np.ones(1)):
    amostras = u.size
    params = 2*n+1
    M = np.ones((amostras, params), dtype=complex)

    for i in range(params):
        M[:,i] = gen_m(u,y,i,p,params)

    M = np.concatenate([M.real, M.imag])
    y_tilde = np.concatenate([y.real, y.imag])

    theta = np.linalg.inv(M.T.dot(M)).dot(M.T).dot(y_tilde)

    y_hat_tilde = np.sum(M*theta.T,axis=1)

    y_hat = y_hat_tilde[:amostras]+1j*y_hat_tilde[amostras:]

    return y_hat, theta

In [23]:
n = 7
for i in range(n):
    print(i,i<n/2,i%2)

0 True 0
1 True 1
2 True 0
3 True 1
4 False 0
5 False 1
6 False 0


In [51]:
n = 5
for i in range(n):
    gen_m(u=np.ones(1), y=np.ones(1), i=i, p=np.ones(n)+0.0001j, params=n)

i: 0, complex: False, real: 1, imag: False, c: True, d: False
i: 1, complex: False, real: 0, imag: True, c: True, d: False
i: 2, complex: False, real: 1, imag: False, c: False, d: True
i: 3, complex: False, real: 0, imag: True, c: False, d: True


In [117]:
from scipy.optimize import root
from functools import partial

In [138]:
from mpmath import findroot, mp

In [254]:
def fun(*p, d, S):
    pr,pi = np.array(p).reshape((2,-1))
    p = pr+1j*pi
    res = 0
    for s in S:
        res+=1 + np.sum(d/(s-p))
    return res

def diff_fun(p, d, S):
    pr,pi = p.reshape((2,-1))
    p = pr+1j*pi
    res = 0
    for s in S:
        res += np.sum(d/(s-p)**2)
    return res

def diff2_fun(*p, d, S):
    res = []
    for s in S:
        res.append(np.sum(d/(s-p)**3))
    return res

S = np.linspace(10,100,20)+np.random.normal(0,1,20)
d = np.ones(5)+np.random.normal(0,0.1,5)
x0 = -5*np.zeros(5)+0.1j+np.random.normal(0,1,5)
x0 = np.concatenate([x0.real, x0.imag])
f = partial(fun, d=d, S=S)
diff_f = partial(diff_fun, d=d, S=S)
diff_f2 = partial(diff_fun, d=d, S=S)


In [252]:
from scipy.optimize import minimize, root, root_scalar

In [255]:
res = minimize(diff_f, x0)

/home/caio/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/home/caio/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:155: ComplexWarning: Casting complex values to real discards the imaginary part
  stp, phi1, derphi1, task = minpack2.dcsrch(alpha1, phi1, derphi1,


In [259]:
root_scalar(f, x0=x0, method='halley', fprime=diff_f, fprime2=diff_f2, maxiter=100000)

RuntimeError: all failed to converge after 100000 iterations

In [237]:
pr,pi = res.x.reshape((2,-1))
p = pr+1j*pi
f(x0)

[(1.5228689639454034+0.005381368529933482j),
 (1.316922631004227+0.0019662595329519557j),
 (1.250644352641464+0.0012283458909807313j),
 (1.2070290810083308+0.0008374979827153786j),
 (1.1735734274187541+0.0005884441970367332j),
 (1.1521902766373175+0.0004522852553241636j),
 (1.1296635069712928+0.0003282331754089331j),
 (1.1182101284105233+0.0002727816885371835j),
 (1.1057921165540543+0.00021846007705110415j),
 (1.0950385618455543+0.00017629247728879514j),
 (1.088212864740841+0.00015187269741724532j),
 (1.0814901576839666+0.00012960135364980917j),
 (1.0756139740499657+0.00011158090394438152j),
 (1.070242716005773+9.628904816545838e-05j),
 (1.0673113335756836+8.841883679802647e-05j),
 (1.0623958975533159+7.597507770203704e-05j),
 (1.0596106540621986+6.934288073404144e-05j),
 (1.057066821682215+6.355023039655027e-05j),
 (1.053453332265344+5.575623952327743e-05j),
 (1.0514296486388761+5.1614046608362166e-05j)]

In [194]:
mp.dps = 15
findroot(f, x0.tolist(), solver='secant')

ValueError: Could not find root within given tolerance. (1.0 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.